<a href="https://colab.research.google.com/github/mualal/statistical-production-forecast/blob/master/notebooks/experiments.ipynb">    <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Открыть и запустить в Google Colaboratory"></a>

In [1]:
# загрузка репозитория, если запуск в Google Colab
try:
    import google.colab, sys
    !git clone https://github.com/mualal/statistical-production-forecast.git
    sys.path.append('statistical-production-forecast')
    %cd statistical-production-forecast
except:
    import sys
    sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# чтение данных из файла
df_initial = pd.read_excel(os.path.abspath(
    os.path.join(
        os.pardir,
        'data',
        'Западно-Чистинное-Ю1(2)-МЭР.xlsx'
    )), sheet_name='МЭР')

# заполнение пустых ячеек нулями
df_initial.fillna(0, inplace=True)
df_initial

,№ скважины,Дата,"Добыча нефти за посл.месяц, т","Добыча жидкости за посл.месяц, т","Время работы в добыче, часы",Объекты работы,Координата забоя Y (по траектории),Координата забоя Х (по траектории)
0,10,2018-07-01,0.00,0.00,0.0,0,6635673,546222
1,10,2018-08-01,0.00,0.00,0.0,0,6635673,546222
2,10,2018-09-01,0.00,0.00,0.0,0,6635673,546222
3,10,2018-10-01,0.00,0.00,0.0,0,6635673,546222
4,10,2018-11-01,0.00,0.00,0.0,0,6635673,546222
...,...,...,...,...,...,...,...,...
1477,9,2021-05-01,197.59,608.68,743.0,Ю1(1),6637374,546191
1478,9,2021-06-01,234.82,663.81,719.0,Ю1(1),6637374,546191
1479,9,2021-07-01,290.16,660.16,743.0,Ю1(1),6637374,546191
1480,9,2021-08-01,239.45,538.77,743.0,Ю1(1),6637374,546191


In [4]:
# отсортированный массив уникальных значений дат
current_date = df_initial['Дата'].unique()
current_date.sort(axis=0)
current_date

array(['2018-07-01T00:00:00.000000000', '2018-08-01T00:00:00.000000000',
       '2018-09-01T00:00:00.000000000', '2018-10-01T00:00:00.000000000',
       '2018-11-01T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2019-01-01T00:00:00.000000000', '2019-02-01T00:00:00.000000000',
       '2019-03-01T00:00:00.000000000', '2019-04-01T00:00:00.000000000',
       '2019-05-01T00:00:00.000000000', '2019-06-01T00:00:00.000000000',
       '2019-07-01T00:00:00.000000000', '2019-08-01T00:00:00.000000000',
       '2019-09-01T00:00:00.000000000', '2019-10-01T00:00:00.000000000',
       '2019-11-01T00:00:00.000000000', '2019-12-01T00:00:00.000000000',
       '2020-01-01T00:00:00.000000000', '2020-02-01T00:00:00.000000000',
       '2020-03-01T00:00:00.000000000', '2020-04-01T00:00:00.000000000',
       '2020-05-01T00:00:00.000000000', '2020-06-01T00:00:00.000000000',
       '2020-07-01T00:00:00.000000000', '2020-08-01T00:00:00.000000000',
       '2020-09-01T00:00:00.000000000', '2020-10-01

In [5]:
# удаление нулевых строк
df_initial = df_initial[
    (df_initial['Добыча нефти за посл.месяц, т'] != 0) &
    (df_initial['Добыча жидкости за посл.месяц, т'] != 0) &
    (df_initial['Время работы в добыче, часы'] != 0) &
    (df_initial['Объекты работы'] != 0)
]
df_initial

,№ скважины,Дата,"Добыча нефти за посл.месяц, т","Добыча жидкости за посл.месяц, т","Время работы в добыче, часы",Объекты работы,Координата забоя Y (по траектории),Координата забоя Х (по траектории)
5,10,2018-12-01,649.18,1290.18,312.0,Ю1(1),6635673,546222
6,10,2019-01-01,1792.89,2174.89,744.0,Ю1(1),6635673,546222
7,10,2019-02-01,1372.47,1564.47,528.0,Ю1(1),6635673,546222
8,10,2019-03-01,2816.75,3518.75,744.0,Ю1(1),6635673,546222
9,10,2019-04-01,2221.11,2608.11,696.0,Ю1(1),6635673,546222
...,...,...,...,...,...,...,...,...
1476,9,2021-04-01,187.51,566.64,720.0,Ю1(1),6637374,546191
1477,9,2021-05-01,197.59,608.68,743.0,Ю1(1),6637374,546191
1478,9,2021-06-01,234.82,663.81,719.0,Ю1(1),6637374,546191
1479,9,2021-07-01,290.16,660.16,743.0,Ю1(1),6637374,546191


In [6]:
# массив уникальных номеров скважин
unique_wells = df_initial['№ скважины'].unique()
unique_wells

array([10, 11, 12, 13, 21, '22В', 24, 25, 30, 31, 33, 35, 36, 37, 38, 39,
       40, 43, 9], dtype=object)

In [7]:
# фактически НИЧЕГО НЕ ДЕЛАЕТ (ясно, что все скважины есть в df_initial)
df_initial = df_initial[df_initial['№ скважины'].isin(unique_wells)]
df_initial

,№ скважины,Дата,"Добыча нефти за посл.месяц, т","Добыча жидкости за посл.месяц, т","Время работы в добыче, часы",Объекты работы,Координата забоя Y (по траектории),Координата забоя Х (по траектории)
5,10,2018-12-01,649.18,1290.18,312.0,Ю1(1),6635673,546222
6,10,2019-01-01,1792.89,2174.89,744.0,Ю1(1),6635673,546222
7,10,2019-02-01,1372.47,1564.47,528.0,Ю1(1),6635673,546222
8,10,2019-03-01,2816.75,3518.75,744.0,Ю1(1),6635673,546222
9,10,2019-04-01,2221.11,2608.11,696.0,Ю1(1),6635673,546222
...,...,...,...,...,...,...,...,...
1476,9,2021-04-01,187.51,566.64,720.0,Ю1(1),6637374,546191
1477,9,2021-05-01,197.59,608.68,743.0,Ю1(1),6637374,546191
1478,9,2021-06-01,234.82,663.81,719.0,Ю1(1),6637374,546191
1479,9,2021-07-01,290.16,660.16,743.0,Ю1(1),6637374,546191


In [8]:
# сортировка сначала по номерам скважин, затем по датам
df_initial = df_initial.sort_values(['№ скважины', 'Дата'])
df_initial

,№ скважины,Дата,"Добыча нефти за посл.месяц, т","Добыча жидкости за посл.месяц, т","Время работы в добыче, часы",Объекты работы,Координата забоя Y (по траектории),Координата забоя Х (по траектории)
1449,9,2019-01-01,817.17,2559.17,456.0,Ю1(1),6637374,546191
1450,9,2019-02-01,1442.18,1689.18,672.0,Ю1(1),6637374,546191
1451,9,2019-03-01,990.38,1287.38,744.0,Ю1(1),6637374,546191
1452,9,2019-04-01,666.01,944.01,696.0,Ю1(1),6637374,546191
1453,9,2019-05-01,473.24,579.24,456.0,Ю1(1),6637374,546191
...,...,...,...,...,...,...,...,...
306,22В,2021-04-01,571.54,733.41,720.0,"Ю1(1), Ю1(2)",6640634,542862
307,22В,2021-05-01,453.66,559.72,529.0,"Ю1(1), Ю1(2)",6640634,542862
308,22В,2021-06-01,546.50,707.29,639.0,"Ю1(1), Ю1(2)",6640634,542862
309,22В,2021-07-01,660.38,860.16,744.0,"Ю1(1), Ю1(2)",6640634,542862


In [9]:
# что-то вроде максимально допустимого промежутка (в днях) между двумя строками измерений в таблице
max_delta = 365
# новый датафрейм
df_new = pd.DataFrame()

# напоминание: loc обращается к имеющемуся pandas индексу; iloc подобен обращению к элементу в массиве

for i in unique_wells:
    # слайс данных для рассматриваемой скважины
    slice = df_initial.loc[df_initial['№ скважины'] == i].copy()
    # пласт (или пласты), на которые работала скважина в последний месяц
    reservoir = slice['Объекты работы'].iloc[-1]
    # история работы только на этот пласт (или эти пласты)
    slice = slice[slice['Объекты работы'] == reservoir]
    # расчёт времени работы (в днях) до следующего измерения
    next_date = slice['Дата'].iloc[1:]
    last_date = slice['Дата'].iloc[-1]
    next_date.loc[-100] = last_date + pd.DateOffset(months=1)
    slice['След. дата'] = np.array(next_date)
    slice['Разница дат'] = slice['След. дата'] - slice['Дата']

    # если скважина работала меньше суток в последний месяц, то он удаляется
    if slice["Время работы в добыче, часы"].iloc[-1] < 24:
        slice = slice.iloc[:-1]
    
    # обрезка истории, если скважина была остановлена больше mаx_delta (или не проводились новые измерения)
    if not slice[slice['Разница дат'] > np.timedelta64(max_delta, 'D')].empty:
        last_index = slice[slice['Разница дат'] > np.timedelta64(max_delta, 'D')].index.to_list()[-1]
        slice = slice.loc[last_index + 1:]
    
    # конкатенация данных по ранее обработанным скважинам с данными по только что обработанной скважине
    df_new = pd.concat([df_new, slice], ignore_index=True)

    # в качестве эксперимента цикл только для одной скважины
    break
print(reservoir)
df_new

Ю1(1)


,№ скважины,Дата,"Добыча нефти за посл.месяц, т","Добыча жидкости за посл.месяц, т","Время работы в добыче, часы",Объекты работы,Координата забоя Y (по траектории),Координата забоя Х (по траектории),След. дата,Разница дат
0,10,2018-12-01,649.18,1290.18,312.0,Ю1(1),6635673,546222,2019-01-01,31 days
1,10,2019-01-01,1792.89,2174.89,744.0,Ю1(1),6635673,546222,2019-02-01,31 days
2,10,2019-02-01,1372.47,1564.47,528.0,Ю1(1),6635673,546222,2019-03-01,28 days
3,10,2019-03-01,2816.75,3518.75,744.0,Ю1(1),6635673,546222,2019-04-01,31 days
4,10,2019-04-01,2221.11,2608.11,696.0,Ю1(1),6635673,546222,2019-05-01,30 days
5,10,2019-05-01,1619.77,1730.77,456.0,Ю1(1),6635673,546222,2019-06-01,31 days
6,10,2019-06-01,2604.46,2756.46,720.0,Ю1(1),6635673,546222,2019-07-01,30 days
7,10,2019-07-01,2576.30,2741.30,744.0,Ю1(1),6635673,546222,2019-08-01,31 days
8,10,2019-08-01,2627.45,2806.45,742.0,Ю1(1),6635673,546222,2019-09-01,31 days
9,10,2019-09-01,2476.97,2672.97,720.0,Ю1(1),6635673,546222,2019-10-01,30 days


In [10]:
# удаление вспомогательных столбцов с разницей дат
df_new.drop(['Разница дат', 'След. дата'], axis=1, inplace=True)
df_new

,№ скважины,Дата,"Добыча нефти за посл.месяц, т","Добыча жидкости за посл.месяц, т","Время работы в добыче, часы",Объекты работы,Координата забоя Y (по траектории),Координата забоя Х (по траектории)
0,10,2018-12-01,649.18,1290.18,312.0,Ю1(1),6635673,546222
1,10,2019-01-01,1792.89,2174.89,744.0,Ю1(1),6635673,546222
2,10,2019-02-01,1372.47,1564.47,528.0,Ю1(1),6635673,546222
3,10,2019-03-01,2816.75,3518.75,744.0,Ю1(1),6635673,546222
4,10,2019-04-01,2221.11,2608.11,696.0,Ю1(1),6635673,546222
5,10,2019-05-01,1619.77,1730.77,456.0,Ю1(1),6635673,546222
6,10,2019-06-01,2604.46,2756.46,720.0,Ю1(1),6635673,546222
7,10,2019-07-01,2576.30,2741.30,744.0,Ю1(1),6635673,546222
8,10,2019-08-01,2627.45,2806.45,742.0,Ю1(1),6635673,546222
9,10,2019-09-01,2476.97,2672.97,720.0,Ю1(1),6635673,546222
